In [6]:
import pandas as pd
from custom_functions_scRBA import *

In [7]:
copy_to_gams = True # if True, copy the model to the gams folder
#### LOAD INPUTS AND PARAMETERS
# Stoichiometry
df_stoich = pd.read_excel('./model/RBA_stoichiometry.xlsx')
df_stoich.index = df_stoich.id.to_list()

# Load protein sequence lengths
df_pro = pd.read_excel('./input/PROTEIN_stoich_curation_2021-09-28.xlsx')
df_pro.index = df_pro.id.to_list()

# Ribosome efficiency (amino acids translated per active ribosome per second)
# (script will convert the unit to per hour) (obsolete / implement directly in GAMS file now)
# kribo = 10.5

# Dummy protein's length (assigned to be the median protein length)
NAA_dummy = 401

In [8]:
### Enzyme synthesis requirement coupled to metabolic reaction rate
idx = df_stoich[df_stoich.coupling_type == 'rxn_enz'].index
eqn_list = []; kapp_list = []; eqn_list_equality = []; kapp_slack_list = []

for i in idx:
    lhs = "v('ENZLOAD-" + df_stoich.id[i][4:] + "') * " + "kapp('" + i + "')"
    print(df_stoich.id[i])
    rhs = "%mu% * v('" + i + "')"
    eqn_list.append(lhs + ' =g= ' + rhs + ';')
    eqn_list_equality.append(lhs + ' =e= ' + rhs + ';')
    kapp_slack_list.append("kapp_slack_ub('" + df_stoich.id[i][4:] + "') - kapp_slack_lb('" + df_stoich.id[i][4:] + "') + " + lhs + ' =e= ' + rhs + ';')
    
eqn_idx = ['EnzCap'+str(i) for i in range(0, len(eqn_list))]
eqn_list = ['EnzCap'+str(i)+'.. ' + eqn_list[i] for i in range(0, len(eqn_list))]

with open('./model/RBA_enzCapacityConstraints_declares.txt', 'w') as f:
    f.write('\n'.join(eqn_idx))
with open('./model/RBA_enzCapacityConstraints_eqns.txt', 'w') as f:
    f.write('\n'.join(eqn_list))
    
eqn_list_equality = ['EnzCap'+str(i)+'.. ' + eqn_list_equality[i] for i in range(0, len(eqn_list_equality))]
with open('./model/RBA_enzCapacityConstraints_eqns_equality_version.txt', 'w') as f:
    f.write('\n'.join(eqn_list_equality))
# kapp slack equations, for testing purposes only (e.g., if kapps are too low, seeing how much they'd need to increase)
kapp_slack_list = ['Equation EnzCap'+str(i)+'; EnzCap'+str(i)+'.. ' + kapp_slack_list[i] for i in range(0, len(kapp_slack_list))]
with open('./model/kapp-slack-RBA_enzCapacityConstraints_equality_version.txt', 'w') as f:
    f.write('\n'.join(kapp_slack_list))

RXN-BTDDH_c_FWD-YAL060W
RXN-BTDDH_c_REV-YAL060W
RXN-13GS_c_FWD-GSC2RHO1
RXN-13GS_c_FWD-GSC1RHO1
RXN-16GS_c_FWD-YPR159W
RXN-16GS_c_FWD-YGR143W
RXN-PRMICI_c_FWD-YIL020C
RXN-P5CD_m_FWD-YHR037W
RXN-P5CD_m_REV-YHR037W
RXN-DHPPDA2_c_FWD-YOL066C
RXN-DRAPPRy_c_FWD-YBR153W
RXN-HPPK2_m_FWD-YNL256W
RXN-AATA_c_FWD-YJL060W
RXN-AATA_c_REV-YJL060W
RXN-AATA_c_FWD-YER152C
RXN-AATA_c_REV-YER152C
RXN-AATA_c_FWD-YGL202W
RXN-AATA_c_REV-YGL202W
RXN-DPR_c_FWD-YHR063C
RXN-IPPMIb_c_FWD-YGL009C
RXN-IPPMIb_c_REV-YGL009C
RXN-IPPS_c_FWD-YOR108W
RXN-UNK3_c_FWD-YHR208W
RXN-UNK3_c_FWD-YJR148W
RXN-UNK3_c_FWD-YHR137W
RXN-UNK3_c_FWD-YGL202W
RXN-HCITR_m_FWD-YDR234W
RXN-HCITR_m_REV-YDR234W
RXN-MCITS_m_FWD-YPR001W
RXN-OMCDC_c_FWD-YJR148W
RXN-OMCDC_m_FWD-YHR208W
RXN-BPNT_c_FWD-YOL064C
RXN-PDE1_c_FWD-YOR360C
RXN-PDE1_c_FWD-YGL248W
RXN-PDE2_c_FWD-YOR360C
RXN-PDE3_c_FWD-YOR360C
RXN-PDE4_c_FWD-YOR360C
RXN-PDE5_c_FWD-YOR360C
RXN-DB4PS_c_FWD-YDR487C
RXN-DHQTi_c_FWD-YDR127W
RXN-DHQS_c_FWD-YDR127W
RXN-3DSPHR_r_FWD-YBR265W
RXN-DDPA_

In [9]:
### Write protein length file (NAA)
pro_list = []
for i in df_pro.index:
    pro_list.append("'PROSYN-" + df_pro.id[i] + "' " + str(len(df_pro.sequence[i]) - 1))
pro_list.append("'PROSYN-PROTDUMMY' " + str(NAA_dummy))

pro_list = ['/'] + pro_list + ['/']
with open('./model/RBA_proteinLength.txt', 'w') as f:
    f.write('\n'.join(pro_list))

In [10]:
### Write prosyn reaction
idx = [i for i in df_stoich.index if i[:7] == 'PROSYN-']
prosyn = ["'" + i + "'" for i in idx]
prosyn = ['/'] + prosyn + ['/']
with open('./model/RBA_rxns_prosyn.txt', 'w') as f:
    f.write('\n'.join(prosyn))

In [11]:
if copy_to_gams: # automatically copies the model folder to the GAMS folder (removing the previous version), if you choose to
    import shutil
    # copy "model" folder to GAMS folder
    # Source folder path
    source_folder = './model'

    # Destination folder path
    destination_folder = '../GAMS/model'

    # Remove the existing destination folder
    shutil.rmtree(destination_folder)

    # Copy the folder
    shutil.copytree(source_folder, destination_folder)